In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

from rubin_sim.data import get_data_dir
import sqlite3

from sat_utils import Constellation, starlink_constellation_v1

In [2]:
dd = get_data_dir()
baseline_file = os.path.join(dd,'sim_baseline/baseline.db')

# Conenct to the sqlite database
con = sqlite3.connect(baseline_file)

# Load up the first year
df = pd.read_sql('select * from observations where night < 366 limit 10;', con)

con.close()

In [3]:
# Make a satellite constellation
tles = starlink_constellation_v1()
constellation = Constellation(tles)

In [4]:
lengths_ns = []
n_rows = len(df)
t1 = time.time()
for index, row in df.iterrows():
    
    lengths_ns.append(constellation.check_pointing(row['altitude'], row['azimuth'],
                                    row['observationStartMJD'], row['visitTime']))
    
    # A simple progress bar
    progress = index/float(n_rows)*100
    text = "\rprogress = %.3f%%" % progress

    sys.stdout.write(text) 
    sys.stdout.flush() 
t2 = time.time()
print('runtime = %f min', ((t2-t1)/60.) )

progress = 90.000%runtime = %f min 1.3940635522206624


In [5]:
# First column is length of streaks in degrees, second is number of satellites making the streaks.
lengths_ns

[(0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0),
 (1.0630268755184136, 1),
 (0.0, 0),
 (0.0, 0),
 (0.0, 0)]

In [6]:

lengths_ns = np.array(lengths_ns)
names = ['observationId', 'streak_length_sum', 'n_streaks']
types = [int, float, int]
result = np.zeros(len(df), dtype=list(zip(names, types)))

result['observationId'] = df['observationId'].values
result['streak_length_sum'] = lengths_ns[:, 0]
result['n_streaks'] = lengths_ns[:, 1]

# And here we would save the result. Hopefully in a file that had a name that made it clear what
# database we used as input


In [7]:
result

array([(0, 0.        , 0), (1, 0.        , 0), (2, 0.        , 0),
       (3, 0.        , 0), (4, 0.        , 0), (5, 0.        , 0),
       (6, 1.06302688, 1), (7, 0.        , 0), (8, 0.        , 0),
       (9, 0.        , 0)],
      dtype=[('observationId', '<i8'), ('streak_length_sum', '<f8'), ('n_streaks', '<i8')])